# This is Step 3 in the Pipeline - Training ML Prediction Model
With this notebook we can train various ML classifiers to tackle multi-lable prediction problem. We are predicting Spec2Vec embeddings from molecular fingerprints.

### Imports

In [1]:
from sklearn.metrics import accuracy_score, f1_score, log_loss, precision_score, recall_score, jaccard_score, roc_auc_score, hamming_loss, label_ranking_loss, coverage_error
from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import  ClassifierChain
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from mass_spectra.similarity_voting import SimilarityVoting
from mass_spectra.nn import NN
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import pickle
from random import shuffle, seed
from math import ceil
import os
from torch.nn import BCEWithLogitsLoss

### Parameters

In [2]:
RANDOM_STATE = 27082023
seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

# path to merged fingerprint and embedding data (fingerprint columns should be prefixed with 'fingerprint_' and embedding columns should be prefixed with 'embedding_').
MERGED_PATH = './source/embedding/all_positive_tms_maccs/merged.csv'
MODEL_OUTPUT_FOLDER = "./source/model/all_positive_tms_maccs/"

In [3]:
assert os.path.isfile(MERGED_PATH)
assert os.path.isdir(MODEL_OUTPUT_FOLDER)
assert MERGED_PATH.endswith('.csv')

In [4]:
ESTIMATOR = None

In [5]:
MODEL = NN(input_size=300, output_size=166, criterion=BCEWithLogitsLoss(), max_epochs=128, batch_size=64, lr=0.1, dropout=0.5)

In [6]:
MODEL_CLASS = MODEL.__class__.__name__
ESTIMATOR_CLASS = ESTIMATOR.__class__.__name__ if ESTIMATOR is not None else 'Multioutput'
MODEL_OUTPUT_FOLDER = f'{MODEL_OUTPUT_FOLDER}{MODEL_CLASS}_{ESTIMATOR_CLASS}'
os.makedirs(f'{MODEL_OUTPUT_FOLDER}/models', exist_ok=False)
os.makedirs(f'{MODEL_OUTPUT_FOLDER}/unseen_inchi_keys_models', exist_ok=False)

### Metrics Definition
Creates metrics which can be called with (y_true, y_prob, y_pred) for easier use. It also creates multiple combinations of metrics for different averaging methods.

In [7]:
Y_PRED_SCORES = [accuracy_score, log_loss, hamming_loss] # input y predictions and y true
Y_PRED_SCORES_WITH_AVERAGING = [f1_score, precision_score, recall_score, jaccard_score] # input y predictions and y true and use one of the following: "micro", "macro", "weighted", "samples"
Y_PROB_SCORES = [roc_auc_score, label_ranking_loss, coverage_error] # input y probabilities and y true

In [8]:
METRICS = []
METRIC_NAMES = []
for metric in Y_PRED_SCORES:
    METRICS.append(lambda y_true, y_prob, y_pred, metric=metric: metric(y_true, y_pred))
    METRIC_NAMES.append(metric.__name__)
for metric in Y_PRED_SCORES_WITH_AVERAGING:
    for average in ["micro", "macro", "weighted", "samples"]:
        zero_division = 0 if metric.__name__ == "jaccard_score" else np.nan
        METRICS.append(lambda y_true, y_prob, y_pred, metric=metric, average=average: metric(y_true, y_pred, average=average, zero_division=zero_division))
        METRIC_NAMES.append(metric.__name__ + "__" + average)
for metric in Y_PROB_SCORES:
    METRICS.append(lambda y_true, y_prob, y_pred, metric=metric: metric(y_true, y_prob))
    METRIC_NAMES.append(metric.__name__)

In [9]:
class Metrics:
    def __init__(self, metrics, metric_names, repeats=2, folds=5):
        self.metrics = metrics
        self.metric_names = metric_names
        
        self.repeats = repeats
        self.folds = folds
        self.i = 0

        self.results = pd.DataFrame(columns=['repeat', 'fold', 'model_training_data_path'] + self.metric_names)
    
    def evaluate(self, y_true, y_prob, y_pred, model_training_data_path=None):
        entry = {
            'repeat': self.i // self.folds,
            'fold': self.i % self.folds,
            'model_training_data_path': model_training_data_path
        }
        for metric, metric_name in zip(self.metrics, self.metric_names):
            try:
                entry[metric_name] = metric(y_true, y_prob, y_pred)
            except ValueError as e:
                print("Warning: ", e)
                entry[metric_name] = np.nan
        
        self.results = pd.concat([self.results, pd.DataFrame(entry, index=[0])], ignore_index=True)
        self.i += 1
    
    def store(self, filename):
        self.results.to_csv(filename, index=False)

    def current(self, metric_name):
        return self.results[metric_name].iloc[-1]

### Load Data

In [10]:
merged_df = pd.read_csv(MERGED_PATH)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025 entries, 0 to 3024
Columns: 467 entries, inchi_key to embedding_299
dtypes: float64(466), object(1)
memory usage: 10.8+ MB


In [11]:
f'Number of NaNs: {merged_df.isna().sum().sum()}' # should be 0

'Number of NaNs: 0'

In [12]:
X = merged_df.filter(regex='^embedding_')
y = merged_df.filter(regex='^fingerprint_')
X.shape, y.shape

((3025, 300), (3025, 166))

In [13]:
X = X.to_numpy()
y = y.to_numpy()

### Train- K-fold Cross Validation

In [14]:
REPEATS = 2
K = 5
metrics = Metrics(METRICS, METRIC_NAMES, REPEATS, K)

for i in tqdm(range(REPEATS), desc="Repeats"):
    kf = KFold(n_splits=K, shuffle=True, random_state=RANDOM_STATE + i)

    for fold, (train_index, test_index) in tqdm(enumerate(kf.split(X, y)), desc="Fold", total=K):
        # train
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        MODEL.fit(X_train, y_train)

        # predict
        y_pred = MODEL.predict(X_test)
        y_prob = MODEL.predict_proba(X_test)

        # store train data
        model_training_data_path = f'{MODEL_OUTPUT_FOLDER}/models/{i}_{fold}.pkl'
        with open(model_training_data_path, "wb") as f:
            pickle.dump({
                "model": MODEL,
                "X_train": X_train,
                "y_train": y_train,
                "X_test": X_test,
                "y_test": y_test,
            }, f)

        # evaluate
        metrics.evaluate(y_test, y_prob, y_pred, model_training_data_path=model_training_data_path)

        # display current results
        print('Label ranking loss: ', metrics.current('label_ranking_loss'))
        print('F1 Weighted: ', metrics.current('f1_score__weighted'))
        
metrics.store(f'{MODEL_OUTPUT_FOLDER}/metrics.csv')

Repeats:   0%|          | 0/2 [00:00<?, ?it/s]

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


Label ranking loss:  0.1957126096152341
F1 Weighted:  0.6162965349188513


c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


...

Label ranking loss:  0.16172769609965665
F1 Weighted:  0.6356421638420565


c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


Label ranking loss:  0.20193833502490371
F1 Weighted:  0.6149007620044423


c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


Label ranking loss:  0.17354537105305862
F1 Weighted:  0.624248455933309


In [15]:
metrics.results.describe()

,accuracy_score,log_loss,hamming_loss,f1_score__micro,f1_score__macro,f1_score__weighted,f1_score__samples,precision_score__micro,precision_score__macro,precision_score__weighted,...,recall_score__macro,recall_score__weighted,recall_score__samples,jaccard_score__micro,jaccard_score__macro,jaccard_score__weighted,jaccard_score__samples,roc_auc_score,label_ranking_loss,coverage_error
count,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,0.0,10.000000,10.000000
mean,0.0,560.106139,0.083043,0.762509,0.138980,0.624172,0.777418,0.945511,0.136585,0.612769,...,0.142339,0.638895,0.674777,0.616202,0.134642,0.607367,0.654046,NaN,0.187438,137.569587
std,0.0,16.434896,0.002482,0.005469,0.000574,0.007510,0.004624,0.002416,0.000407,0.007549,...,0.000769,0.007685,0.006583,0.007160,0.000826,0.007447,0.006148,NaN,0.012815,4.131536
min,0.0,530.678835,0.078522,0.755347,0.138464,0.614901,0.770942,0.942799,0.136128,0.603494,...,0.141508,0.628868,0.664355,0.606873,0.133864,0.597952,0.644728,NaN,0.161728,129.358678
25%,0.0,555.029365,0.082460,0.759127,0.138670,0.620052,0.774997,0.943896,0.136301,0.608414,...,0.141859,0.634990,0.672914,0.611769,0.134191,0.602450,0.651094,NaN,0.178918,135.631405
50%,0.0,564.299629,0.083556,0.762108,0.138729,0.622990,0.777078,0.944846,0.136525,0.610197,...,0.142055,0.638397,0.674712,0.615651,0.134255,0.606638,0.653698,NaN,0.192205,136.477686
75%,0.0,565.681160,0.084176,0.763991,0.139247,0.625895,0.779075,0.946320,0.136690,0.613841,...,0.142840,0.641036,0.676832,0.618111,0.134944,0.609436,0.656474,NaN,0.195954,140.838017
max,0.0,581.721675,0.086110,0.772081,0.140263,0.637742,0.785868,0.949926,0.137435,0.626815,...,0.143967,0.652357,0.686693,0.628772,0.136562,0.620521,0.665171,NaN,0.201938,143.433058


### Train With Unseen InChI Keys

In [16]:
def split_dataset(X, y, test_inchi_keys=[]):
    # get index from merged_df
    test_index = merged_df[merged_df['inchi_key'].isin(test_inchi_keys)].index
    train_index = merged_df[~merged_df['inchi_key'].isin(test_inchi_keys)].index

    # split X and y
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    return X_train, X_test, y_train, y_test

In [17]:
all_inchi_keys = list(merged_df['inchi_key'].unique())
shuffle(all_inchi_keys)

In [18]:
hidden_inchi_keys = 10

REPEATS = 1
K = ceil(len(all_inchi_keys) / hidden_inchi_keys)
metrics = Metrics(METRICS, METRIC_NAMES, REPEATS, K)

for i in tqdm(range(REPEATS), desc="Repeats"):
    # Reshuffle
    shuffle(all_inchi_keys)

    for end_i in tqdm(range(hidden_inchi_keys, len(all_inchi_keys), hidden_inchi_keys), desc="Fold", total=K):
        start_i = end_i - hidden_inchi_keys
        if end_i + hidden_inchi_keys > len(all_inchi_keys):
            end_i = len(all_inchi_keys)

        # train
        test_inchi_keys = all_inchi_keys[start_i:end_i]
        X_train, X_test, y_train, y_test = split_dataset(X, y, test_inchi_keys)

        MODEL.fit(X_train, y_train)

        # predict
        y_pred = MODEL.predict(X_test)
        y_prob = MODEL.predict_proba(X_test)

        # store train data
        model_training_data_path = f'{MODEL_OUTPUT_FOLDER}/unseen_inchi_keys_models/{start_i}_{end_i}.pkl'
        with open(model_training_data_path, "wb") as f:
            pickle.dump({
                "model": MODEL,
                "X_train": X_train,
                "y_train": y_train,
                "X_test": X_test,
                "y_test": y_test,
            }, f)

        # evaluate
        metrics.evaluate(y_test, y_prob, y_pred, model_training_data_path=model_training_data_path)

        # display current results
        print('Label ranking loss: ', metrics.current('label_ranking_loss'))
        print('F1 Weighted: ', metrics.current('f1_score__weighted'))

metrics.store(f'{MODEL_OUTPUT_FOLDER}/unseen_inchi_keys_metrics.csv')

Repeats:   0%|          | 0/1 [00:00<?, ?it/s]

Fold:   0%|          | 0/11 [00:00<?, ?it/s]

c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


Label ranking loss:  0.20436720100329747
F1 Weighted:  0.654548592956313


c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


...

c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


Label ranking loss:  0.09855701454726253
F1 Weighted:  0.7378228215656275


c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


Label ranking loss:  0.14139083080352455
F1 Weighted:  0.7073151183608404
Label ranking loss:  0.26140200518255374
F1 Weighted:  0.5885538780228561


c:\Users\aleks\anaconda3\envs\mass_spectra\lib\site-packages\sklearn\metrics\_classification.py:2916: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(


In [19]:
metrics.results.describe()

,accuracy_score,log_loss,hamming_loss,f1_score__micro,f1_score__macro,f1_score__weighted,f1_score__samples,precision_score__micro,precision_score__macro,precision_score__weighted,...,recall_score__macro,recall_score__weighted,recall_score__samples,jaccard_score__micro,jaccard_score__macro,jaccard_score__weighted,jaccard_score__samples,roc_auc_score,label_ranking_loss,coverage_error
count,10.0,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,0.0,10.000000,10.000000
mean,0.0,563.190917,0.084795,0.761249,0.138786,0.626491,0.773822,0.936991,0.135796,0.614799,...,0.143685,0.642794,0.673559,0.617037,0.134585,0.611752,0.648539,NaN,0.197944,140.727701
std,0.0,133.867624,0.022731,0.051026,0.004151,0.064975,0.049891,0.038611,0.005494,0.067598,...,0.004275,0.061967,0.060592,0.067706,0.005307,0.068890,0.067154,NaN,0.055466,15.361597
min,0.0,397.419886,0.051071,0.696315,0.132569,0.547328,0.714791,0.871237,0.127361,0.542456,...,0.138554,0.549088,0.597952,0.534113,0.127361,0.535359,0.574168,NaN,0.098557,106.885350
25%,0.0,468.288001,0.069932,0.726283,0.136882,0.576482,0.734563,0.912530,0.132057,0.559446,...,0.139738,0.596538,0.624073,0.570238,0.131145,0.558888,0.593863,NaN,0.164520,133.560767
50%,0.0,547.990758,0.085203,0.751584,0.138494,0.613058,0.761054,0.929966,0.135707,0.598538,...,0.143674,0.635850,0.663028,0.602075,0.133850,0.594499,0.629033,NaN,0.195011,144.145695
75%,0.0,640.668958,0.098057,0.793907,0.141317,0.668998,0.805928,0.968114,0.136947,0.654188,...,0.146487,0.688059,0.718321,0.658336,0.136918,0.653203,0.692566,NaN,0.244793,151.519784
max,0.0,834.193404,0.120289,0.846659,0.145776,0.737823,0.861558,0.990298,0.145904,0.736891,...,0.150320,0.739410,0.776111,0.734092,0.143238,0.733570,0.770005,NaN,0.268579,156.528701
